# Market Basket Analysis

## Description

Market Basket Analysis (MBA) is a data mining technique that is used to uncover purchase patterns in any retail setting. <br>
The goal of Market Basket Analysis is to understand consumer behavior by identifying relationships between the items that people buy. 

# Libraries

In [27]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing
pd.options.mode.chained_assignment = None
import datetime as dt
from math import sqrt
from mlxtend.frequent_patterns import apriori, association_rules # analysis

import csv
import re

import matplotlib.pyplot as plt 
import seaborn as sns

from google.cloud import bigquery

# Data Universe
Definição do cenário base:

>  * Análise de transações de todas as lojas LM

> * Período tempo:
    * 2021 <br>

>* Considerar todas as transações (tickets) que contém pelo menos um artigo da secção 7 (Sanitário)

> * Incidência da análise será sobre todo o tipo de clientes;


> * Análise desenvolvida em diferentes granularidades para conseguirmos aferir a que produz insigths mais relevantes (Familia de produto, produto, etc …):


> * Compras online e loja física;


## Variables Description

Ticket information
________________________
    'DAT_VTE' - Purchase date 
    'NUM_TIC' - Ticket ID
    'NUM_RGRPCLI' - Client ID
    'LIB_TYPFID_CART' - Type of fidelity card
    'NUM_PERSLM_ENT_CRE' - ID of who made the sale 
        (to distinguish between in-store and online purchase; equal to 99999999 or 999016 means online; otherwise in-store purchase)
    'MNT_TTCDEVETT' - Amount spent

Store
________________________
    'NUM_ETT' - Store ID
    'LIB_MAG' - Store name
    'NUM_REG' - Store region ID
    'LIB_REG' - Store region name
    'NUM_RGRP' - Store format ID
    'LIB_RGRP' - Store format name
    
Item
________________________
    'NUM_ART' - Item ID
    'LIB_ART' - Item name
    'NUM_RAY' - Section ID
    'LIBNUMRAY' - Section name
    'NUM_SRAY' - Subsection ID
    'LIBCODSRAY' - Subsection name
    'NUM_TYP' - Type ID
    'LIBCODTYP' - Type name
    'NUM_STYP' - Subtype ID
    'LIBCODSTYP' - Subtype name

In [28]:
def make_null_table(data):
    """ Return the information about missing values in data.
    
    """
    null_columns = (data.isnull().sum(axis = 0)/len(data)).sort_values(ascending=False).index
    null_data = pd.concat([
    data.isnull().sum(axis = 0),
    (data.isnull().sum(axis = 0)/len(data)).sort_values(ascending=False),
    data.loc[:, data.columns.isin(list(null_columns))].dtypes], axis=1)
    null_data = null_data.rename(columns={0: '# null', 
                                          1: '% null', 
                                          2: 'type'}).sort_values(ascending=False, by = '% null')
    null_data = null_data[null_data["# null"]!=0]
    
    return null_data

In [29]:
def get_stores(bq_table):
    client = bigquery.Client()
    query = """
        SELECT DISTINCT(NUM_ETT) 
        FROM `{}`
        ORDER BY 1
        """.format(bq_table)
    query_job = client.query(query=query)
    results = query_job.to_dataframe()

    return list(results['NUM_ETT'])

In [30]:
def get_data(bq_table, store):
    """ Return the respective sales dataset from stores.
    
    """
    client = bigquery.Client()
    query = """
                SELECT * 
                FROM `{}`
                WHERE NUM_ETT = {}
                """.format(bq_table,store)
    query_job = client.query(query=query)
    dataset = query_job.to_dataframe()
    
    return dataset

In [31]:
def get_data_online(bq_table):
    """ Return the respective sales dataset from stores.
    
    """
    client = bigquery.Client()
    query = """
                SELECT * 
                FROM `{}`
                WHERE NUM_PERSLM_ENT_CRE IN (99999999,999016)
                """.format(bq_table)
    query_job = client.query(query=query)
    dataset = query_job.to_dataframe()
    
    return dataset

In [32]:
def conversions(data, var_type = []):
    """ Converts all 'int64' variables to 'object' type and specified conversions. Returns dataset with converted variables.
    
    Parameters
    ----------
    data : dataset
        The input sales dataset.
    var_type : list of lists with len 2
        [variable, type] list.
    
    """
    cols_to_change = data.select_dtypes(include='int64').columns
    data[cols_to_change] = data[cols_to_change].astype('object')

    for var in var_type:
        data[var[0]] = data[var[0]].astype(var[1])
        
    return data

In [33]:
def filter_rows_by_products(data, col, products):
    """ Return dataset without the specified products (e.g. plastic bags) 
    
    Parameters
    ----------
    data : dataset
        The input sales dataset.
    col : string
        The column name for the products variable.
    products : list
        List of products to remove.
    
    """
    return data[~data[col].isin(products)]

In [34]:
def preprocessing(data):
    """ Basic DS preprocessing.
    
    """
    # Variables definition
    #global df_final_nodups
    #global types_purchase
    #global lib_regs
    #global lib_rgrps
    #global stores
    
    # Keep non-negative values (exclude returns)
    data = data[data.MNT_TTCDEVETT >= 0]
    
    # Add variable (online vs in-store purchase)
    data['TYPE_PURCHASE'] = np.where(
        data['NUM_PERSLM_ENT_CRE'].isin([99999999, 999016]), 'ONLINE', 'IN-STORE') 
    
    # Composing an identifier for transactions - by date, store, and ticket number
    data.loc[:, 'DAT_VTE'] = data['DAT_VTE'].astype('str')
    data.loc[:, 'NUM_TIC'] = data['NUM_TIC'].astype('str')
    data.loc[:, 'NUM_ETT'] = data['NUM_ETT'].astype('str')
    data.loc[:, 'ETT_DAT_TIC'] = data['NUM_ETT'] + '-' + data['DAT_VTE'] + ' - ' + data['NUM_TIC']
    
    # Select tickets with at least one product from section 7 (Bathroom)
    list_tickets = data[data['NUM_RAY']==7]['ETT_DAT_TIC']
    df_final = data[data['ETT_DAT_TIC'].isin(list_tickets)].reset_index(drop=True)
    
    # Drop duplicates
    df_final_nodups = df_final.drop_duplicates(keep='first', subset=['DAT_VTE','NUM_TIC','NUM_RAY','NUM_SRAY','NUM_TYP','NUM_STYP','NUM_ART','LIB_ART'], inplace=False)
    
    # Create lists of purchase types, regions, and store types - these will be needed for running iterations in MBA
    types_purchase = df_final_nodups['TYPE_PURCHASE'].unique()
    lib_regs = df_final_nodups['LIB_REG'].unique()
    lib_rgrps = df_final_nodups['LIB_RGRP'].unique()
    stores = df_final_nodups['NUM_ETT'].unique()
    
    # Get subjects (transactions) and slices (type/subtype of item) 
    df_final_nodups.loc[:, 'QTY'] = 1
    df_final_nodups.loc[:, 'QTY'] = df_final_nodups['QTY'].astype('uint8')
    df_final_nodups.loc[:, 'NUM_TYP'] = df_final_nodups['NUM_TYP'].astype('str')
    df_final_nodups.loc[:, 'NUM_STYP'] = df_final_nodups['NUM_STYP'].astype('str')
    df_final_nodups.loc[:, 'NUM_RAY'] = df_final_nodups['NUM_RAY'].astype('str')
    df_final_nodups.loc[:, 'NUM_SRAY'] = df_final_nodups['NUM_SRAY'].astype('str')
    df_final_nodups.loc[:, 'DAT_VTE'] = df_final_nodups['DAT_VTE'].astype('str')
    df_final_nodups.loc[:, 'NUM_TIC'] = df_final_nodups['NUM_TIC'].astype('str')
    df_final_nodups.loc[:, 'NUM_ETT'] = df_final_nodups['NUM_ETT'].astype('str')
    
    # Add variables
    df_final_nodups['Year'] = pd.DatetimeIndex(df_final_nodups['DAT_VTE']).year
    
    # Define the naming of each slice
    df_final_nodups.loc[:, 'TYP'] = df_final_nodups['NUM_RAY'] + '/' + df_final_nodups['NUM_SRAY'] + '/' + df_final_nodups['NUM_TYP'] + ': ' + df_final_nodups['LIBCODTYP']
    
    # Define the naming of each transaction
    df_final_nodups.loc[:, 'DAT_NUM_TIC'] = df_final_nodups['NUM_ETT'] + '-' + df_final_nodups['DAT_VTE'] + ' - ' + df_final_nodups['NUM_TIC']
    
    return df_final_nodups

In [35]:
def do_mba(basket,type_purchase,store_number):

    basket_slice = basket[(basket.TYPE_PURCHASE == type_purchase) & (basket.NUM_ETT == store_number)]

    nr_transac_items = basket_slice.shape[0]
    lib_reg = basket_slice['LIB_REG'].iloc[0]
    lib_rgrp = basket_slice['LIB_RGRP'].iloc[0]
    store_name = basket_slice['LIB_MAGCRT'].iloc[0]
    year = basket_slice['Year'].iloc[0]

    # printing number of items
    print('Transaction items for',type_purchase,store_number,':',nr_transac_items)
    
    # only advance if number of items is larger than zero
    if nr_transac_items > 0:
        calc_support = 0.001
        
        print('Applying apriory algorithm with minimum support =', calc_support)
        
        # getting data ready for apriori - this will produce a matrix with t rows and p columns
        # t is the number of subjects - transactions
        # p is the number of slices - product type/subtype
        basket_df_final_nodups = (basket_slice.groupby(['DAT_NUM_TIC', 'TYP'])['QTY'].max().unstack(fill_value=0).reset_index().set_index('DAT_NUM_TIC'))
        
        # getting t from the matrix
        nr_transacs = basket_df_final_nodups.shape[0]
        
        # run the apriori algorithm, inputting the matrix, min support, and setting some basic parameters
        df_final_frq_items = apriori(basket_df_final_nodups, min_support = calc_support , use_colnames = True, low_memory = True)
        
        # print the shape of the itemset dataframe
        print('Shape after apriori for',type_purchase,store_number,':',df_final_frq_items.shape,'\n')
        
        # only advance if number of itemsets is larger than zero
        if df_final_frq_items.shape[0] > 0:
            # run the association_rules algorithm to get the rules
            # input the itemset dataframe obtained by the aprori algorithm, and set parameters for minimum *quality* of rules
            rules = association_rules(df_final_frq_items, metric="lift", min_threshold=1)

            rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])
            
            # post-algorithm filtering
            rules_filtered = rules[(rules.confidence > 0.20) & (rules.conviction > 1)]

            rules_filtered['transactions'] = rules_filtered['support']*nr_transacs
            rules_filtered['transactions'] = rules_filtered['transactions'].astype('int')
            rules_filtered['total_transactions'] = nr_transacs
            rules_filtered['LIB_REG'] = lib_reg
            rules_filtered['LIB_RGRP'] = lib_rgrp
            rules_filtered['NUM_ETT'] = store_number
            rules_filtered['LIB_MAGCRT'] = store_name
            rules_filtered['year'] = year
            rules_filtered.rename({'antecedent support': 'antecedent_support', 'consequent support': 'consequent_support'}, axis=1, inplace=True)

            # these steps are necessary to avoid antecedent and consequent itemsets being represented in the csv as:
            # {frozenset: slgaslkn}
            rules_filtered['antecedents'] = rules_filtered['antecedents'].apply(lambda x: list(x)[0:]).astype("unicode")
            rules_filtered['consequents'] = rules_filtered['consequents'].apply(lambda x: list(x)[0:]).astype("unicode")
            
            rules_filtered['transactions_antecedents'] = rules_filtered['antecedent_support']*nr_transacs
            rules_filtered['transactions_antecedents'] = rules_filtered['transactions_antecedents'].astype('int')
            rules_filtered['transactions_consequents'] = rules_filtered['consequent_support']*nr_transacs
            rules_filtered['transactions_consequents'] = rules_filtered['transactions_consequents'].astype('int')

            rules_filtered['antecedents_section'] = rules_filtered['antecedents'].str.findall(r"(?<=')\d+").str.join('; ')            
            rules_filtered['antecedents_subsection'] = rules_filtered['antecedents'].str.findall(r"(?<=/)\d+(?=/)").str.join('; ')
            rules_filtered['antecedents_type'] = rules_filtered['antecedents'].str.findall(r"(?<=/)\d+(?=:)").str.join('; ')

            shapebeforefilt = rules.shape
            shapeafterfilt = rules_filtered.shape
            
            # filter rules containing at least one product of section 7
            regex = "\'7"
            antecedents_7 = rules_filtered['antecedents']
            df_antecedents = antecedents_7.str.contains(regex)
            consequents_7 = rules_filtered['consequents']
            df_consequents = consequents_7.str.contains(regex)
            rules_final = rules_filtered[(df_antecedents) | (df_consequents)]
            rules_filtered = rules_final
            
            rules_filtered['suppconf']=rules_filtered['support']*rules_filtered['confidence']
            
            columns_to_include = ['year','LIB_RGRP','LIB_REG','NUM_ETT','LIB_MAGCRT',
                                  'antecedents_section','antecedents_subsection','antecedents_type','antecedents',
                                  'consequents','transactions',
                                  'confidence','antecedent_support','consequent_support',
                                  'support','lift','conviction','suppconf','total_transactions',
                                  'transactions_antecedents','transactions_consequents',]
            
            rules_filtered = rules_filtered[columns_to_include]
            rules_filtered = rules_filtered.sort_values('suppconf', ascending=False)
            
            filename = 'TYP07_RULES_' + type_purchase + '_' + str(year)
            
            if store_number=='1':
                # delete existing table and create a new one
                save_files(rules_filtered,'data-test-lab-lmpt.BaseTimestampLmPt',filename,'WRITE_TRUNCATE')
                
                # create new notes csv file
                csv_file_notas = open(filename + '_notas', 'w')
            
            else:
                save_files(rules_filtered,'data-test-lab-lmpt.BaseTimestampLmPt',filename,'WRITE_APPEND')
                
                # append notes in csv file
                csv_file_notas = open(filename + '_notas', 'a')

            #filename = 'TYP07_RULES_' + type_purchase + '_STORE_' + store_number + '_' + str(year)
            #rules_filtered.to_gbq('BaseTimestampLmPt.'+filename,'data-test-lab-lmpt',if_exists='replace')
            #save_files(rules_filtered,'data-test-lab-lmpt.BaseTimestampLmPt',filename)

            

            
            # write in __notas
            csv_file_notas.write('Rules for \n Purchase type: ' + type_purchase + '\n Year: ' + str(year) + '\n Region: ' + lib_reg + '\n Store type: ' + lib_rgrp + '\n Store number: ' + store_number  + '\n Store name: ' + store_name)
            csv_file_notas.write('\n Total number of transactions: ' + str(nr_transacs))
            csv_file_notas.write('\n Transaction items: ' + str(nr_transac_items) + '\n Applying apriori algorithm with minimum support = ' + str(calc_support) )
            csv_file_notas.write('\n Shape after apriori: ' + str(shapebeforefilt) + '\n Shape of rules with metric conditions: ' + str(shapeafterfilt)
                                + '\n Shape of filtered rules: ' + str(rules_filtered.shape) +'\n')
            csv_file_notas.write('--------------------------------------------------------------')
            csv_file_notas.write('\n')

            csv_file_notas.close()                      

        else:
            # transactions in the slice but no itemsets after apriori
            print('No rules were calculated because the basket is empty')
    else:
        # no transactions in the slice
        print('No transactions, nothing was done')

In [36]:
def do_mba2(basket, type_purchase, store_number=None, calc_support = 0.001):

    if type_purchase=='ONLINE':
        basket_slice = basket[(basket.TYPE_PURCHASE == type_purchase)]

        nr_transac_items = basket_slice.shape[0]
        year = basket_slice['Year'].iloc[0]
    
        # printing number of items
        print('Transaction items for',type_purchase,':',nr_transac_items)
        
    else:
        basket_slice = basket[(basket.TYPE_PURCHASE == type_purchase) & (basket.NUM_ETT == store_number)]

        nr_transac_items = basket_slice.shape[0]
        lib_reg = basket_slice['LIB_REG'].iloc[0]
        lib_rgrp = basket_slice['LIB_RGRP'].iloc[0]
        store_name = basket_slice['LIB_MAGCRT'].iloc[0]
        year = basket_slice['Year'].iloc[0]
    
        # printing number of items
        print('Transaction items for',type_purchase,store_number,':',nr_transac_items)
    
    # only advance if number of items is larger than zero
    if nr_transac_items > 0:
        
        print('Applying apriori algorithm with minimum support =', calc_support)
        
        # getting data ready for apriori - this will produce a matrix with t rows and p columns
        # t is the number of subjects - transactions
        # p is the number of slices - product type/subtype
        basket_df_final_nodups = (basket_slice.groupby(['DAT_NUM_TIC', 'TYP'])['QTY'].max().unstack(fill_value=0).reset_index().set_index('DAT_NUM_TIC'))
        
        # getting t from the matrix
        nr_transacs = basket_df_final_nodups.shape[0]
        
        # run the apriori algorithm, inputting the matrix, min support, and setting some basic parameters
        df_final_frq_items = apriori(basket_df_final_nodups, min_support = calc_support , use_colnames = True, low_memory = True)

        # print the shape of the itemset dataframe
        if type_purchase=='ONLINE':
            print('Shape after apriori for',type_purchase,':',df_final_frq_items.shape,'\n')
        else:
            print('Shape after apriori for',type_purchase,store_number,':',df_final_frq_items.shape,'\n')
        
        # only advance if number of itemsets is larger than zero
        if df_final_frq_items.shape[0] > 0:
            # run the association_rules algorithm to get the rules
            # input the itemset dataframe obtained by the aprori algorithm, and set parameters for minimum *quality* of rules
            rules = association_rules(df_final_frq_items, metric="lift", min_threshold=1)

            rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])
            
            # post-algorithm filtering
            rules_filtered = rules[(rules.confidence > 0.20) & (rules.conviction > 1)]

            rules_filtered['transactions'] = rules_filtered['support']*nr_transacs
            rules_filtered['transactions'] = rules_filtered['transactions'].astype('int')
            rules_filtered['total_transactions'] = nr_transacs
            rules_filtered['year'] = year

            if type_purchase!='ONLINE':
                rules_filtered['LIB_REG'] = lib_reg
                rules_filtered['LIB_RGRP'] = lib_rgrp
                rules_filtered['NUM_ETT'] = store_number
                rules_filtered['LIB_MAGCRT'] = store_name
                
            rules_filtered.rename({'antecedent support': 'antecedent_support', 'consequent support': 'consequent_support'}, axis=1, inplace=True)

            # these steps are necessary to avoid antecedent and consequent itemsets being represented in the csv as:
            # {frozenset: slgaslkn}
            rules_filtered['antecedents'] = rules_filtered['antecedents'].apply(lambda x: list(x)[0:]).astype("unicode")
            rules_filtered['consequents'] = rules_filtered['consequents'].apply(lambda x: list(x)[0:]).astype("unicode")
            
            rules_filtered['transactions_antecedents'] = rules_filtered['antecedent_support']*nr_transacs
            rules_filtered['transactions_antecedents'] = rules_filtered['transactions_antecedents'].astype('int')
            rules_filtered['transactions_consequents'] = rules_filtered['consequent_support']*nr_transacs
            rules_filtered['transactions_consequents'] = rules_filtered['transactions_consequents'].astype('int')

            rules_filtered['antecedents_section'] = rules_filtered['antecedents'].str.findall(r"(?<=')\d+").str.join('; ')            
            rules_filtered['antecedents_subsection'] = rules_filtered['antecedents'].str.findall(r"(?<=/)\d+(?=/)").str.join('; ')
            rules_filtered['antecedents_type'] = rules_filtered['antecedents'].str.findall(r"(?<=/)\d+(?=:)").str.join('; ')

            shapebeforefilt = rules.shape
            shapeafterfilt = rules_filtered.shape
            
            # filter rules containing at least one product of section 7
            regex = "\'7"
            antecedents_7 = rules_filtered['antecedents']
            df_antecedents = antecedents_7.str.contains(regex)
            consequents_7 = rules_filtered['consequents']
            df_consequents = consequents_7.str.contains(regex)
            rules_final = rules_filtered[(df_antecedents) | (df_consequents)]
            rules_filtered = rules_final
            
            rules_filtered['suppconf']=rules_filtered['support']*rules_filtered['confidence']
            
            
            if type_purchase=='ONLINE':
                columns_to_include = ['year',
                                  'antecedents_section','antecedents_subsection','antecedents_type','antecedents',
                                  'consequents','transactions',
                                  'confidence','antecedent_support','consequent_support',
                                  'support','lift','conviction','suppconf','total_transactions',
                                  'transactions_antecedents','transactions_consequents',]
            
                rules_filtered = rules_filtered[columns_to_include]
                rules_filtered = rules_filtered.sort_values('suppconf', ascending=False)

                filename = 'TYP07_RULES_' + type_purchase + '_' + str(year)

                # delete existing table and create a new one
                save_files(rules_filtered,'data-test-lab-lmpt.BaseTimestampLmPt',filename,'WRITE_TRUNCATE')

                # create new notes csv file
                csv_file_notas = open(filename + '_notas', 'w')

                # write in __notas
                csv_file_notas.write('Rules for \n Purchase type: ' + type_purchase + '\n Year: ' + str(year))
                csv_file_notas.write('\n Total number of transactions: ' + str(nr_transacs))
                csv_file_notas.write('\n Transaction items: ' + str(nr_transac_items) + '\n Applying apriori algorithm with minimum support = ' + str(calc_support) )
                csv_file_notas.write('\n Shape after apriori: ' + str(shapebeforefilt) + '\n Shape of rules with metric conditions: ' + str(shapeafterfilt)
                                    + '\n Shape of filtered rules: ' + str(rules_filtered.shape) +'\n')
                csv_file_notas.write('--------------------------------------------------------------')
                csv_file_notas.write('\n')

                csv_file_notas.close()    
                
            else: # IN-STORE
                columns_to_include = ['year','LIB_RGRP','LIB_REG','NUM_ETT','LIB_MAGCRT',
                                      'antecedents_section','antecedents_subsection','antecedents_type','antecedents',
                                      'consequents','transactions',
                                      'confidence','antecedent_support','consequent_support',
                                      'support','lift','conviction','suppconf','total_transactions',
                                      'transactions_antecedents','transactions_consequents',]


                rules_filtered = rules_filtered[columns_to_include]
                rules_filtered = rules_filtered.sort_values('suppconf', ascending=False)

                filename = 'TYP07_RULES_' + type_purchase + '_' + str(year)

                if store_number=='1':
                    # delete existing table and create a new one
                    save_files(rules_filtered,'data-test-lab-lmpt.BaseTimestampLmPt',filename,'WRITE_TRUNCATE')

                    # create new notes csv file
                    csv_file_notas = open(filename + '_notas', 'w')

                else:
                    save_files(rules_filtered,'data-test-lab-lmpt.BaseTimestampLmPt',filename,'WRITE_APPEND')

                    # append notes in csv file
                    csv_file_notas = open(filename + '_notas', 'a')

                # write in __notas
                csv_file_notas.write('Rules for \n Purchase type: ' + type_purchase + '\n Year: ' + str(year) + '\n Region: ' + lib_reg + '\n Store type: ' + lib_rgrp + '\n Store number: ' + store_number  + '\n Store name: ' + store_name)
                csv_file_notas.write('\n Total number of transactions: ' + str(nr_transacs))
                csv_file_notas.write('\n Transaction items: ' + str(nr_transac_items) + '\n Applying apriori algorithm with minimum support = ' + str(calc_support) )
                csv_file_notas.write('\n Shape after apriori: ' + str(shapebeforefilt) + '\n Shape of rules with metric conditions: ' + str(shapeafterfilt)
                                    + '\n Shape of filtered rules: ' + str(rules_filtered.shape) +'\n')
                csv_file_notas.write('--------------------------------------------------------------')
                csv_file_notas.write('\n')

                csv_file_notas.close()                      

        else:
            # transactions in the slice but no itemsets after apriori
            print('No rules were calculated because the basket is empty')
    else:
        # no transactions in the slice
        print('No transactions, nothing was done')

In [37]:
def save_files(input_dataframe, bq_dataset, bq_table, write_disposition='WRITE_TRUNCATE'):
    """ Saves tables into BigQuery project.
    
    Parameters
    ----------
    input_dataframe : dataframe
        The input dataframe.
    bq_dataset : string
        The Dataset ID from BigQuery.
    bq_table: string
        The name for the new table.
    write_disposition : string
        Specifies the action that occurs if destination table already exists.
        The default value is WRITE_TRUNCATE.
        Possible values:
            WRITE_APPEND - If the table already exists, BigQuery appends the data to the table.
            WRITE_EMPTY - If the table already exists and contains data, a ‘duplicate’ error is returned in the job result.
            WRITE_TRUNCATE - If the table already exists, BigQuery overwrites the table data.
            
    """
    # Load client
    client = bigquery.Client()

    # Write output 
    destination = bigquery.Dataset(bq_dataset).table(bq_table)
    job_config = bigquery.LoadJobConfig(
        write_disposition = write_disposition,
    )
    job = client.load_table_from_dataframe(
            input_dataframe, 
            destination, 
            job_config=job_config
    )  # Make an API request.
    job.result()

In [40]:
def setup(data, store_number=None): 
    #var, type tuple
    vars_to_convert = [
        ['DAT_VTE','datetime64[ns]'],
        ['NUM_RGRPCLI','object'],
        ['MNT_TTCDEVETT','float64']
    ]
    data = conversions(data, vars_to_convert)
    
    filter_products = ['SACO PLASTICO LEVE UNIDADE','SACO PAPEL UNI']
    data = filter_rows_by_products(data, 'LIB_ART', filter_products)
    
    basket = preprocessing(data)

    if store_number:
        do_mba2(basket,'IN-STORE',str(store_number))
    else:
        #do_mba2(basket,'ONLINE',calc_support = 0.007)
        do_mba2(basket,'ONLINE')

In [24]:
def main():
    bq_table='data-test-lab-lmpt.BaseTimestampLmPt.mba_2021'
    
    #store_arr = [1,9] #1,9,59
    store_arr = get_stores(bq_table)
    
    # MBA per store
    for store in store_arr:
        df = get_data(bq_table,store)    
        setup(df,store)
    
    # MBA for online
    df_online = get_data_online(bq_table)
    setup(df_online)

In [14]:
if __name__ == '__main__':
    main()

Transaction items for IN-STORE 1 : 287090
Applying apriori algorithm with minimum support = 0.001
Shape after apriori for IN-STORE 1 : (2301, 2) 

Transaction items for IN-STORE 2 : 677041
Applying apriori algorithm with minimum support = 0.001
Shape after apriori for IN-STORE 2 : (3845, 2) 

Transaction items for IN-STORE 3 : 516685
Applying apriori algorithm with minimum support = 0.001
Shape after apriori for IN-STORE 3 : (3521, 2) 

Transaction items for IN-STORE 4 : 311577
Applying apriori algorithm with minimum support = 0.001
Shape after apriori for IN-STORE 4 : (3778, 2) 

Transaction items for IN-STORE 5 : 724364
Applying apriori algorithm with minimum support = 0.001
Shape after apriori for IN-STORE 5 : (4318, 2) 

Transaction items for IN-STORE 6 : 316860
Applying apriori algorithm with minimum support = 0.001
Shape after apriori for IN-STORE 6 : (2301, 2) 

Transaction items for IN-STORE 7 : 370181
Applying apriori algorithm with minimum support = 0.001
Shape after apriori 

In [1]:
# Download all files
#!zip -r MBA_20220502_allfiles.zip *